<a href="https://colab.research.google.com/github/Ang3lino/recomenderSys/blob/master/collaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np
import pandas as pd

import os
import pickle

from sortedcontainers import SortedList
from collections import Counter, defaultdict
from tqdm import tqdm  # modulo cuya finalidad es dar un feedback del progreso de algun procedimiento

In [21]:

!pip install tqdm

# User-Based Neighborhood Models
En este enfoque, user-based neighborhoods son definidos con el fin de identificar usuario similares a el usuario objetivo con el fin de predecir valoraciones que el daria a un articulo.

Para la matrix de valoraciones $m \times n$, $R = [r_{ui}]$ con $m$ usuarios y $n$ articulos. Definamos $I_u$ el conjunto de indices de articulos que el usuario $u$ ha valorado (una fila). El conjunto $I_u \cap I_v$ define las observaciones mutuamente observadas.

In [5]:
from google.colab import drive  
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_object(fname: str, user_count: int, item_count: int) -> defaultdict:
    fdir = 'drive/My Drive/petroleo/movielens-20m-dataset'
    fname = f'{fname}_{user_count}_{item_count}.json'
    fpath = os.path.join(fdir, 'shrinked', fname)
    with open(fpath, 'rb') as fp:
        object_ = pickle.load(fp)
    return object_

def defaultdict_set(defdict):
    return {k: set(v) for k, v in defdict.items()}


user_count = 4096
item_count = 512
user2item = load_object('user2item', user_count, item_count)
item2user = load_object('item2user', user_count, item_count)
user_item2rating = load_object('user_item2rating', user_count, item_count)

user2item = defaultdict_set(user2item)

# Pearson
Una medida que captura la similitud $Sim(u, v)$ entre los vectores de valoracion entre usuarios $u,v$ es el coeficiente de correlacion de Pearson. El primer paso es calcular las medias de valoracion $\mu_u$ para cada usuario $u$:

$$ \mu_u = \frac{\sum_{k\in I_u}r_{uk}}{|I_u|} $$

Asi, la correlacion de Parson se define como 

$$ Sim(u, v) = Pearson(u, v) = \frac{\sum_{k \in I_u \cap I_v}(r_{uk} - \mu_u)(r_{vk} - \mu_v)}
{\sqrt{\sum_{k \in I_u \cap I_v}(r_{uk} - \mu_u)^2 }  \sqrt{ \sum_{k \in I_u \cap I_v}(r_{vk} - \mu_v)^2 } 
} $$



In [0]:
def user_means(user_item2rating: dict, user2item: defaultdict) -> dict:
    means = dict() 
    for u, items in user2item.items():
        means[u] = np.mean([user_item2rating[(u, i)] for i in items])
    return means

def sim_cos(u: np.array, v: np.array):
    dot = lambda u, v: u.dot(v)
    sigma = lambda u, v: np.sqrt(dot(u, v))
    num = dot(u, v)
    den = sigma(u, u) * sigma(v, v)
    return num / den
    

def sim_pearson(u: int, v: int, means: dict, user2item: defaultdict, user_item2rating: dict):
    intersection = user2item[u] & user2item[v]
    a = np.array([user_item2rating[u, k] for k in intersection]) - means[u]
    b = np.array([user_item2rating[v, k] for k in intersection]) - means[v]
    return sim_cos(a, b)



In [26]:
means = user_means(user_item2rating, user2item)
ans = sim_pearson(1, 5, means, user2item, user_item2rating)
print(ans)

0.11996713555118083
